In [1]:
cd ..

/Users/dreamgonfly/Documents/pozalabs/infinityWord


In [2]:
from datasets.seq2seq_datasets import Seq2SeqIndexedDataset

In [3]:
from models.encoders.rnn_encoder import GRUEncoder
from models.decoders.rnn_decoder import GRUDecoder
from models.rnn_model import RNNModel

In [4]:
from trainers.seq2seq_trainer import Seq2SeqTrainer, seq2seq_collate_fn

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

In [6]:
from utils import get_logger

In [7]:
from dictionaries import BaseDictionary

In [8]:
dic = BaseDictionary.load('base_dictionary')

In [9]:
embeddings = nn.Embedding(num_embeddings=50000, embedding_dim=128)
encoder = GRUEncoder(embeddings, hidden_size=256)
decoder = GRUDecoder(embeddings, hidden_size=256)
model = RNNModel(encoder, decoder)
logger = get_logger(log_name='gru_encoder_decoder')
trainer = Seq2SeqTrainer(model=model, 
                               train_dataloader=DataLoader(Seq2SeqIndexedDataset('train'), batch_size=32, collate_fn=seq2seq_collate_fn), 
                               val_dataloader=DataLoader(Seq2SeqIndexedDataset('val'), batch_size=32, collate_fn=seq2seq_collate_fn),
                               loss_function=CrossEntropyLoss(size_average=False),
                               optimizer=Adam(model.parameters()),
                               device=torch.device('cpu'),
                               logger=logger)

In [10]:
trainer.run(epochs=10)

  0%|          | 0/817 [00:00<?, ?it/s]

torch.Size([32, 19, 50000])
train_targets_flat torch.Size([608])


KeyboardInterrupt: 

In [11]:
gru = nn.GRU(input_size=100, hidden_size=200)

In [12]:
gru??

Signature:   gru(*input, **kwargs)
Type:        GRU
String form: GRU(100, 200)
File:        ~/anaconda3/lib/python3.6/site-packages/torch/nn/modules/rnn.py
Source:     
class GRU(RNNBase):
    r"""Applies a multi-layer gated recurrent unit (GRU) RNN to an input sequence.


    For each element in the input sequence, each layer computes the following
    function:

    .. math::

            \begin{array}{ll}
            r_t = \sigma(W_{ir} x_t + b_{ir} + W_{hr} h_{(t-1)} + b_{hr}) \\
            z_t = \sigma(W_{iz} x_t + b_{iz} + W_{hz} h_{(t-1)} + b_{hz}) \\
            n_t = \tanh(W_{in} x_t + b_{in} + r_t (W_{hn} h_{(t-1)}+ b_{hn})) \\
            h_t = (1 - z_t) n_t + z_t h_{(t-1)} \\
            \end{array}

    where :math:`h_t` is the hidden state at time `t`, :math:`x_t` is the input
    at time `t`, :math:`h_{(t-1)}` is the hidden state of the previous layer
    at time `t-1` or the initial hidden state at time `0`, and :math:`r_t`,
    :math:`z_t`, :math:`n_t` are the reset, 

In [11]:
%debug

> /Users/dreamgonfly/Documents/pozalabs/infinityWord/trainers/seq2seq_trainer.py(50)train()
     48             vocabulary_size = train_decoder_outputs.size(-1)
     49             train_outputs_flat = train_decoder_outputs.view(-1, vocabulary_size)
---> 50             train_targets_flat = train_targets.view(-1)
     51             train_batch_loss = self.loss_function(train_outputs_flat, train_targets_flat)
     52 



ipdb>  train_targets.view(-1)


*** RuntimeError: invalid argument 2: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Call .contiguous() before .view(). at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten/src/TH/generic/THTensor.cpp:280


ipdb>  q


In [13]:
import torch

In [14]:
from torch.nn.functional import softmax

In [15]:
from dictionaries import BaseDictionary

In [16]:
dictionary = BaseDictionary.load('base_dictionary')

In [17]:
inputs = torch.tensor([dictionary['<StartSent>']]).unsqueeze(0)
inputs

tensor([[ 2]])

In [18]:
model.eval()

LSTM(
  (embedding): Embedding(50000, 128, padding_idx=0)
  (lstm): LSTM(128, 1024, num_layers=2, batch_first=True, dropout=0.5)
  (decoder): Linear(in_features=1024, out_features=50000, bias=True)
)

In [19]:
dictionary.idx2word[softmax(model(inputs), dim=2).argmax().item()]

'탄탄대로의'

In [20]:
from generators import OneSentenceGenerator

In [21]:
generator = OneSentenceGenerator(model, dictionary, checkpoint_filepath='parameters/LSTM/LSTM_0_2018-07-28 19:31:26.237748.pth')

In [23]:
generator.generate_one()

'머리에는 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'